In [1]:
import pickle
import numpy as np
import pandas as pd
import csv


In [14]:
#df = pd.read_csv("/Users/alexandra/Documents/data/tapas/tapas_100_000_simra_format.csv", nrows = 100000)
#df.to_csv("/Users/alexandra/Documents/data/tapas/tapas_100_000lines_simra_format.csv", index=False)

In [2]:
used_words = pickle.load(open("output/simra/no_dp/cell_1000/used_words_1000_epsNone.pickle", "rb"))

In [3]:
#traces = pickle.load(open("output/generated_traces-cell_1000-eps_100000.00_10.pickle", "rb"))

In [4]:
orig_init = pickle.load(open("/Users/alexandra/Documents/GitHub/DP-Loc/output/simra/no_dp/cell_1000/orig_init_data-cell_1000.pickle", "rb"))


In [5]:
orig_init

array([[148., 124.,  11.],
       [ 14.,  61.,  19.],
       [ 58.,  35.,  18.],
       ...,
       [183.,  65.,  10.],
       [ 47., 191.,  14.],
       [  4., 111.,  18.]], dtype=float32)

In [3]:
traces = pickle.load(open("/Users/alexandra/Documents/GitHub/DP-Loc/output/Geolife/EPS_NONE/generated_traces-cell_632-eps_5.00_100.pickle", "rb"))

In [6]:
traces = pickle.load(open("/Users/alexandra/Documents/GitHub/DP-Loc/output/simra/no_dp/cell_1000/generated_traces-cell_1000-eps_None_100.pickle", "rb"))

In [4]:
seq = np.array(traces)[:,2]


/var/folders/m7/jb7xk8h91p541g43zv2r8l740000gn/T/ipykernel_14078/127097865.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  seq = np.array(traces)[:,2]


In [5]:
unique_values, counts = np.unique([len(np.unique(l)) for l in seq], return_counts = True)

In [6]:
unique_values

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10])

In [7]:
counts

array([  192, 16157,  5827,  2398,   731,   159,    41,     6,     1])

In [8]:
np.array([len(s) for s in seq]).max()

32

In [9]:
# output traces
seq

array([list([1, 1, 60, 60, 290, 290, 290, 290]),
       list([75, 75, 117, 117, 117, 290, 290, 290]),
       list([111, 111, 181, 181, 181, 181, 290, 290, 290, 290]), ...,
       list([3, 3, 78, 78, 256, 256, 256, 256]),
       list([149, 149, 347, 347, 347, 347, 417, 417, 417, 417]),
       list([261, 261, 261, 261, 324, 324, 53, 53, 53, 53, 31, 31, 31, 31])],
      dtype=object)

In [10]:
from utils import *
cnf = load_cfg("cfg/cfg_general_Simra.json")

# create a projector class, as in the algorithm
projector = Projector(cnf.MIN_X, cnf.MIN_Y, cnf.MAX_X, cnf.MAX_Y)

MAP_HEIGHT = int(ceil(projector.toProjected(cnf.MAX_X, cnf.MAX_Y)[1] / cnf.CELL_SIZE))
MAP_WIDTH = int(ceil(projector.toProjected(cnf.MAX_X, cnf.MAX_Y)[0] / cnf.CELL_SIZE))

# dicts for conversion as in projector class
cells = [(x, y) for x in range(MAP_HEIGHT + 1) for y in range(MAP_WIDTH + 1)]
cell2id = dict(zip(cells, range(1, len(cells) + 1)))
id2cell = dict(zip(range(1, len(cells) + 1), cells))

# you need to get the used words in the algorithm
# I added the following line to the preproc.py file in line 170:
# pickle.dump(used_words, open(f"output/simra/used_words_{cnf.CELL_SIZE}_eps{cnf.EPS}.pickle", "wb"))

used_words = pickle.load(open("/Users/alexandra/Documents/GitHub/DP-Loc/output/simra/used_words_500_eps100000.pickle", "rb"))

# dicts for conversion of token and cells
cell2token = dict(zip(used_words, range(1, len(used_words) + 1)))
token2cell = {v: k for k, v in cell2token.items()}

2022-12-19 17:25:59.594181: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
# get the generated traces
#traces = pickle.load(open("output/simra/generated_traces-cell_500-eps_100000.00_5.pickle", "rb"))
seq = np.array(traces)[:,2]

# write these as readable and mapped csv file
with open("/Users/alexandra/Documents/GitHub/synth_data_evaluation/data/synthetic/dploc/dploc_cell_500_eps_100000.csv", 'w') as csvfile: 
    write = csv.writer(csvfile) 
    write.writerow(["tid", "lng", "lat", "datetime"])
    counter = 0
    for i in range(0, len(traces)-1):
        ids = [id2cell[token2cell[cell_id]] for cell_id in traces[i][2]]
        coords_projected = [(y*cnf.CELL_SIZE, x*cnf.CELL_SIZE) for x,y in ids]
        coords_gps = list(projector.toGPS_list(coords_projected))
        for coord_pair in coords_gps:
            x, y = coord_pair
            write.writerow([i, x, y, traces[i][1]]) #TODO: now the same timestamp for each point in a trajectory is used. Maybe convert this to a timestamp and make it increasing?
            counter += 1 


/var/folders/m7/jb7xk8h91p541g43zv2r8l740000gn/T/ipykernel_57239/2868893615.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  seq = np.array(traces)[:,2]


In [12]:
#words = []
#for word_counts in used_words:
#    words.append(word_counts[0])

In [13]:
ids = []
for i in range(0, len(seq)-1):
    ids = ids + [token2cell[token] for token in seq[i]]

In [14]:
grid_ids_output = pd.Series(ids)

In [15]:
grid_ids_output.value_counts()[1:10]        #.sort_index().plot(kind="bar")

1375    9211
852     2655
1024    2039
641     1984
636     1840
694     1342
1019    1105
844     1023
1461     886
dtype: int64

In [16]:
from tables import *

h5file = open_file("/Users/alexandra/Documents/GitHub/DP-Loc/datasets/simra/mapped_data-304.h5", mode="r")
input_traces = h5file.root.Traces
total_grams = sum([len(trace) for trace in input_traces])
print("Total grams:", total_grams)
print("Total traces:", len(input_traces))
#h5file.close()

Total grams: 394553
Total traces: 21648


In [17]:
len(input_traces)

21648

In [18]:
input_traces[1:10]

[[[1025, 1602870240],
  [1026, 1602870300],
  [1028, 1602870360],
  [986, 1602870420],
  [945, 1602870480],
  [946, 1602870540],
  [947, 1602870600],
  [948, 1602870660],
  [949, 1602870720],
  [949, 1602870780],
  [950, 1602870840],
  [951, 1602870900],
  [995, 1602870960],
  [995, 1602871020],
  [1038, 1602871080],
  [1039, 1602871140],
  [1040, 1602871200],
  [998, 1602871260],
  [999, 1602871320],
  [957, 1602871380],
  [958, 1602871440],
  [959, 1602871500],
  [961, 1602871560],
  [920, 1602871620],
  [922, 1602871680],
  [922, 1602871740],
  [923, 1602871800]],
 [[493, 1592756640],
  [492, 1592756700],
  [492, 1592756760],
  [491, 1592756820],
  [533, 1592756880],
  [575, 1592756940],
  [617, 1592757000],
  [617, 1592757060],
  [660, 1592757120],
  [702, 1592757180],
  [744, 1592757240],
  [743, 1592757300],
  [784, 1592757360],
  [784, 1592757420],
  [826, 1592757480],
  [826, 1592757540],
  [868, 1592757600],
  [909, 1592757660],
  [951, 1592757720],
  [951, 1592757780],
  [994

In [19]:
import numpy as np
trace_length = [len(trace) for trace in input_traces]

In [20]:
print(min(trace_length))
print(max(trace_length))
print(np.mean(trace_length))
print(np.median(trace_length))

2
182
18.225840724316335
16.0


In [21]:
ids_input = []
for i in range(0, len(input_traces)-1):
    for j in range(0, len(input_traces[i])-1):    
        ids_input.append(input_traces[i][j][0])

grid_ids_input = pd.Series(ids_input)

In [23]:
h5file.close()

In [24]:
# grid_ids_input.value_counts()[1:10]                 #.sort_index().plot(kind="bar")

In [22]:
# grid input to coordinates
with open("/Users/alexandra/Documents/GitHub/synth_data_evaluation/data/synthetic/dploc/simra_input_sampled40_gramsize10_reversed.csv", 'w') as csvfile: 
    write = csv.writer(csvfile) 
    write.writerow(["tid", "lng", "lat"])
    for i in range(0, len(input_traces)-1):
        ids = [id2cell[cell_id[0]] for cell_id in input_traces[i]]
        coords_projected = [(y*cnf.CELL_SIZE, x*cnf.CELL_SIZE) for x,y in ids]
        coords_gps = list(projector.toGPS_list(coords_projected))
        for coord_pair in coords_gps:
            x, y = coord_pair
            write.writerow([i, x, y]) 

In [24]:
# all cords in mapping
all_coords_input = [(x*250, y*250) for x,y in cell2id.keys()]
all_coords = list(projector.toGPS_list(all_coords_input))

In [25]:
import csv
with open("/Users/alexandra/Documents/GitHub/synth_data_evaluation/data/synthetic/dploc/grid_coords_simra.csv", 'w') as csvfile: 
    write = csv.writer(csvfile) 
    write.writerow(["lng", "lat"])
    for coords in all_coords:
        write.writerow(coords)  